# Predicting Shaking With Machine Learning

This project uses two different Machine Learning techniques to predict shaking

- Linear Regression
- Regression Trees



In [ ]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [ ]:
csv_file = 'shaking.csv'
df = pd.read_csv(csv_file)
df2 = pd.read_csv(csv_file)
df.describe()

In [ ]:
#columns=list(df.columns)
#for column in columns[2:]:
#    new_name='norm_'+column
#    df2[column]=(df[column]-df[column].min())/(df[column].max()-df[column].min())

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='RdBu')

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
from IPython.display import Image
Image(url= "https://miro.medium.com/max/750/1*TSX7fu85EwGEdnhA-Sv4cA.jpeg", width=700, height=700)

In [ ]:
print(df.iloc[0])
print(df.iloc[1])

In [ ]:
auX = df[['Ml','epicentral_distance']]
X = np.array(auX)
auY = df[['channel_pgv_ms']]
y = np.array(auY)
y = np.log10(y)
print("X's shape is:", X.shape)
print("y's shape is:", y.shape)
newY=np.zeros(302)
newX=np.zeros((302,2))
counter=0
#print(indexPos)
for x in range(0, len(df), 2):
    if (y[x]>y[x+1]):
        newY[counter]=y[x]
        newX[counter]=X[x]
    else:
        newY[counter]=y[x+1]
        newX[counter]=X[x+1]  
    counter+=1

In [ ]:
# Because the data is low dimensional we can visualize all of it.  
# In general, this won't be the case so you should get used to summary statistics.
plt.figure(figsize=(10,8))
sc = plt.scatter(df['epicentral_distance'].values, df['Ml'].values, c=np.log10(df['channel_pgv_ms'].values), cmap=plt.cm.get_cmap('jet'),edgecolors='gray')
cbar = plt.colorbar(sc)
plt.title("$log_{10}$(PGV) vs. Distance and Magnitude")
plt.ylabel("Ml")
plt.xlabel("Distance (km)")
cbar.set_label('$log_{10}$(PGV)')#, rotation=270)
plt.xlim(0,250)
plt.grid()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.2, random_state=58317)
reg = LinearRegression(fit_intercept=True) 
reg.fit(X_train, y_train) 

In [ ]:
# (3) Compare with mean-squared error of the test and training dataset
y_predict_train = reg.predict(X_train) # Computes estimates using the trained model
y_predict_test  = reg.predict(X_test)  # Computes estimates using the trained model
mse_train = np.sum((y_train - y_predict_train)**2)/len(y_predict_train)
mse_test  = np.sum((y_test  -  y_predict_test)**2)/len(y_predict_test)  # or mean_squared_error(y_predict, y_test)
print("Training MSE:", mse_train)
print("Testing MSE:", mse_test)
print("Proportion of variance explained in training data (R^2):", reg.score(X_train,  y_train)) 
print("Proportion of variance explained in test data (R^2):", reg.score(X_test,  y_test))

In [ ]:
print("Predicted log10(PGV) = %lf %+lf*Ml %+lf*epicentral_distance"%(reg.intercept_, reg.coef_[0], reg.coef_[1]))

In [ ]:
# Because the data is low dimensional we can visualize all of it.  
# In general, this won't be the case so you should get used to summary statistics.
plt.figure(figsize=(10,8))
sc = plt.scatter(X_test[:,1], X_test[:,0], c=y_test - y_predict_test, cmap=plt.cm.get_cmap('RdBu'),edgecolors='gray')
cbar = plt.colorbar(sc)
plt.title("Test Set Residual $log_{10}$(PGV) vs. Distance and Magnitude")
plt.ylabel("Ml")
plt.xlabel("Distance (km)")
cbar.set_label('Residual $log_{10}$(PGV)')#, rotation=270)
plt.xlim(0,250)
plt.clim(-1,1)
plt.grid()
plt.show()

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()

X_train, X_test, y_train, y_test = train_test_split(newX, newY, test_size=0.2, random_state=58317)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

from sklearn import metrics
#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
#print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# Because the data is low dimensional we can visualize all of it.  
# In general, this won't be the case so you should get used to summary statistics.
plt.figure(figsize=(10,8))
sc = plt.scatter(X_test[:,1], X_test[:,0], c=y_test - y_pred, cmap=plt.cm.get_cmap('RdBu'),edgecolors='gray')
cbar = plt.colorbar(sc)
plt.title("Test Set Residual $log_{10}$(PGV) vs. Distance and Magnitude")
plt.ylabel("Ml")
plt.xlabel("Distance (km)")
plt.clim(-1,1)
cbar.set_label('Residual $log_{10}$(PGV)')#, rotation=270)
plt.xlim(0,250)
plt.clim(-1, 1);
plt.grid()
plt.show()

In [ ]:
mse_train = np.sum((y_train - y_predict_train)**2)/len(y_predict_train)
mse_test  = np.sum((y_test  -  y_pred)**2)/len(y_pred)  # or mean_squared_error(y_predict, y_test)
#print(np.shape(y_train))
#print(np.shape(y_predict_train))
#print(np.shape(y_pred))
print("Training MSE:", mse_train)
print("Testing MSE:", mse_test)
print('R^2 (coefficient of determination) regression score function:',metrics.r2_score(y_test, y_pred))

In [ ]:
importance=clf.feature_importances_
objects = ('Magnitude', 'PGV')
y_pos = np.arange(len(objects))
performance = [10,8]
plt.bar(y_pos, importance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Feature importance')
plt.show()

# Takeaways
-Straightforward, easy to use and explain techniques

-Trees are high-variance, can improve tree model with Boosting

-Need more data

